# Retrieve と RetrieveAndGenerate の違い

Knowledge bases for Amazon Bedrock を利用した検索には 2 種類の API が提供されています。

- [RetrieveAndGenerate](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_RetrieveAndGenerate.html)
- [Retrieve](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_Retrieve.html)

文字通り Retrieve API はベクトルデータベースから情報の取得のみを行い、 RetrieveAndGenerate API はベクトルデータベースから情報の取得した後、回答生成まで行います。

![](../images/02/01.png)

![](../images/02/02.png)

RetrieveAndGenerate API と Retrieve API の挙動を確認してみます。

## RetrieveAndGenerate API

カーネルは `base` を選択してください。

![](../images/02/03.png)

In [ ]:
import boto3

KNOWLEDGEBASE_ID = "" # ナレッジベース ID を記載
model_arn = "arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0"
bedrock_agent = boto3.client("bedrock-agent-runtime")
prompt = "私はエンジニアです。オンボーディング初日に必要なことを教えてください。"

response = bedrock_agent.retrieve_and_generate(
    input={"text": prompt},
    retrieveAndGenerateConfiguration={
        "type": 'KNOWLEDGE_BASE',
        "knowledgeBaseConfiguration": {
            "knowledgeBaseId": KNOWLEDGEBASE_ID,
            "modelArn": model_arn
        },
    },
)

print(response["output"]["text"])

## Retrieve API

In [ ]:
response = bedrock_agent.retrieve(
    knowledgeBaseId=KNOWLEDGEBASE_ID,
    retrievalQuery={
        "text": prompt
    },
    retrievalConfiguration={
            'vectorSearchConfiguration': {
            'numberOfResults': 2
        }
    }
)

retrieval_results = response.get("retrievalResults")
print(retrieval_results)

## Retrieve して Generate してみよう

このままだと素っ気ないため、Retrieve した結果を元に回答を生成してみましょう。

Retrieve API の醍醐味は、回答生成に利用するモデルを自由に選択できることです。

今回は RetrieveAndGenerate API (Knowledge bases for Amazon Bedrock) でサポートされていない `Claude 3 Opus` を回答生成に使ってみましょう。

[Converse API](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html) を利用することで、共通化された API スキーマでモデルを実行できます。

In [ ]:
# ベクトルデータベースへ検索
response = bedrock_agent.retrieve(
    knowledgeBaseId=KNOWLEDGEBASE_ID,
    retrievalQuery={
        "text": prompt
    },
    retrievalConfiguration={
            'vectorSearchConfiguration': {
            'numberOfResults': 2
        }
    }
)
retrieval_results = response.get("retrievalResults")

# コンテキストを突っ込む箱の作成
messages = []
bedrock = boto3.client("bedrock-runtime")
model_id = "anthropic.claude-3-opus-20240229-v1:0"

# 質問文の整形
user_inputs = {
    "role": "user",
    "content": [{"text": prompt}],
}
messages.append(user_inputs)

# ナレッジベースからの回答を整形
assistant_inputs = {
    "role": "assistant",
	"content": [],
}

for text in retrieval_results:
    assistant_inputs.get("content").append({"text": text.get("content").get("text")})
messages.append(assistant_inputs)

# 回答文の生成
response = bedrock.converse(
    modelId=model_id,
    messages=messages,
)
answer = response.get("output").get("message").get("content")[0].get("text")
print(answer)

RetrieveAndGenerate API は、ベクトルデータベースからの情報取得と回答文の生成を1つの API で行う、非常に便利なインターフェースです。

しかし、回答生成に利用可能なモデルには制限があるため、複数のモデルを使用してモデルを比較したい場合は、 Retrieve API を利用して、独自に回答生成処理を実装する方法が効果的です。

## まとめ

このセクションでは次のことを学びました。

- Retrieve と RetrieveAndGenerate の違い
- ナレッジベースを利用した回答生成には 2 種類のパターンがある
  - RetrieveAndGenerate API で一貫した回答生成
  - Retrieve と Generate を分けて API 実行
    - 利用可能なモデルの選択肢を広げる
- Converse API はモデルごとに異なる API スキーマを抽象化してくれる

Next: [ハイブリッド検索について体験してみよう](./03.ipynb)